In [ ]:
"""
Evaluator–Optimizer Workflow (상세페이지 자동 생성용)
---------------------------------------------------
- LangGraph StateGraph + 명시적 State 타입(TypedDict)
- Generator / Evaluator (Feedback 흡수) 2-노드 구조
- 비용 최적화: gpt-4o-mini
- 프롬프트 외부 .md 파일 분리
- 조건부 전이 + (필요 시) Command/Send로 직접 재호출
"""

from typing import TypedDict, Optional, Dict, Any
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, Send
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
import json, os


# =========================================================
# 모델 설정 (비용 효율)
# =========================================================
GEN_MODEL = "gpt-4o-mini"
EVAL_MODEL = "gpt-4o-mini"

llm_generator = ChatOpenAI(model=GEN_MODEL, temperature=0.7)
llm_evaluator = ChatOpenAI(model=EVAL_MODEL, temperature=0.3)

# =========================================================
# 헬퍼 - 프롬프트 로더
# =========================================================
GEN_PROMPT_PATH = "./prompts/generator_prompt.md"
EVAL_PROMPT_PATH = "./prompts/evaluator_prompt.md"

def load_prompt(path: str) -> str:
    if not os.path.exists(path):
        raise FileNotFoundError(f"Prompt file not found: {path}")
    with open(path, "r", encoding="utf-8") as f:
        return f.read().strip()
        

# =========================================================
# 1) State 정의
# =========================================================
class State(TypedDict):
    meta: Dict[str, Any]                  # 제품 메타정보 (입력)
    draft: Optional[str]                  # Generator 결과 (초안)
    evaluation: Optional[Dict[str, Any]]  # Evaluator JSON 평가 결과
    feedback: Optional[str]               # 수정 지시사항(누적/최신)
    iteration: int                        # 루프 카운트

# =========================================================
# 2) 평가 결과 스키마
# =========================================================
class EvalResult(BaseModel):
    score: float = Field(..., ge=0, le=100)
    pass_: bool = Field(..., alias="pass")
    revise_instructions: list[str] = []
    violations: list[dict] = []
    
# =========================================================
# 1) 노드: Generator
# =========================================================
def node_generate(state: State) -> State:
    meta = state["meta"]
    feedback = state.get("feedback") or "초안 1회차: 기본 작성"
    base_prompt = load_prompt(GEN_PROMPT_PATH)

    prompt = base_prompt.format(
        product_name=meta["product_name"],
        target_audience=meta["target_audience"],
        seo_keywords=", ".join(meta["seo_keywords"]),
        certs_or_tests=meta.get("certs_or_tests", "없음"),
        banned_terms=", ".join(meta.get("banned_terms", [])),
        feedback=feedback,
    )

    resp = llm_generator.invoke(prompt)
    return {**state, "draft": resp.content, "iteration": state["iteration"] + 1}

# =========================================================
# 2) 노드: Evaluator (피드백 생성까지 수행)
#    - 조건: 통과/최대루프 → 상태만 반환 (조건부 전이로 END)
#    - 조건: 미통과 → Command/Send로 Generator 재호출 (Feedback 별도 노드 불필요)
# =========================================================
def node_evaluate(state: State):
    draft = state["draft"]
    meta = state["meta"]
    eval_prompt_base = load_prompt(EVAL_PROMPT_PATH)

    eval_prompt = eval_prompt_base.format(
        product_name=meta["product_name"],
        draft=draft,
    )

    resp = llm_evaluator.invoke(eval_prompt)

    try:
        parsed = EvalResult.parse_raw(resp.content)
        evaluation = parsed.dict(by_alias=True)
    except Exception:
        evaluation = {
            "score": 0,
            "pass": False,
            "revise_instructions": ["JSON 형식 오류로 인한 재시도 필요"],
            "violations": [],
        }

    # Evaluator가 피드백 생성까지 수행
    feedback_lines = evaluation.get("revise_instructions", [])
    feedback_text = "\n".join(feedback_lines) if feedback_lines else None

    updated: State = {
        **state,
        "evaluation": evaluation,
        "feedback": feedback_text,
    }

    # 통과 또는 최대 루프 → 상태만 반환(그래프 조건부 전이로 END)
    if (evaluation.get("pass") and evaluation.get("score", 0) >= 90) or (state["iteration"] >= 3):
        return updated

    # 미통과 → Command/Send를 사용해 즉시 Generator 재호출 (Feedback 노드 불필요)
    return Command(
        sends=[
            Send("Generator", updated)
        ]
    )

# =========================================================
# 2) 상태 전이 조건 (Evaluator 이후 전이만 정의)
#    - node_evaluate에서 Command/Send로 재호출한 경우 조건 전이는 무시됨
#    - 상태만 반환된 경우에만 조건 전이가 적용
# =========================================================
def route_after_eval(state: State) -> str:
    eval_res = state.get("evaluation") or {}
    if (eval_res.get("pass") and eval_res.get("score", 0) >= 90) or (state.get("iteration", 0) >= 3):
        return "Accepted"
    return "Rejected+Feedback"  # 일반적으로 여기 도달하지 않음(미통과 시 evaluator가 Command/Send 사용)
    
# =========================================================
# 그래프 빌드
# =========================================================
graph_builder = StateGraph(State)

graph_builder.add_node("Generator", node_generate)
graph_builder.add_node("Evaluator", node_evaluate)

graph_builder.add_edge(START, "Generator")
graph_builder.add_edge("Generator", "Evaluator")
graph_builder.add_edge("Evaluator", END)

# Evaluator가 상태만 반환(accept/maxed)한 경우에만 END로 전이
graph_builder.add_conditional_edges(
    "Evaluator",
    route_after_eval,
    {
        "Accepted": END,
        "Rejected+Feedback": "Generator",  # 안전망(보통 미사용) — evaluator가 Command/Send 실패 시 사용
    },
)

graph = graph_builder.compile()


# graph

meta_info = {
    "product_name": "AEMICA 스테인리스 수세미 세트",
    "target_audience": "주부 및 자취생",
    "seo_keywords": ["스테인리스 수세미", "주방 청소", "위생 세척"],
    "certs_or_tests": "KC 인증 완료",
    "banned_terms": ["완벽", "기적", "100%"],
}

initial_state: State = {
    "meta": meta_info,
    "draft": None,
    "evaluation": None,
    "feedback": None,
    "iteration": 0,
}

result  = graph.invoke(initial_state)
